# Clasificación de imágenes usando una red convolucional

## Entrenamiento de una red VGG16

In [1]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image


import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

from bcd.utils.paths import data_raw_dir, models_dir

In [2]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 20
VALIDATION_SPLIT = 0.2

In [3]:
# Función para obtener todos los archivos de imagen en un directorio
def get_files(directory):
    all_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                all_files.append(os.path.join(root, file))
    return all_files

# Obtener todas las imágenes y sus etiquetas
train_dir = data_raw_dir("train/")
all_files = get_files(train_dir)
labels = [int(os.path.basename(os.path.dirname(f))) for f in all_files]

In [4]:
# Dividir los datos en conjuntos de entrenamiento y validación
train_files, val_files, train_labels, val_labels = train_test_split(
    all_files, labels, test_size=VALIDATION_SPLIT, stratify=labels, random_state=42)

### Crear generadores de datos

In [5]:
# Crear generadores de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
def generate_from_paths_and_labels(file_paths, labels, batch_size, img_generator, target_size, shuffle=True):
    while True:
        batch_paths, batch_labels = train_test_split(file_paths, test_size=batch_size, stratify=labels)
        batch_images = []
        for path in batch_paths:
            img = tf.keras.preprocessing.image.load_img(path, color_mode='grayscale', target_size=target_size)
            img = tf.keras.preprocessing.image.img_to_array(img)
            img = img_generator.random_transform(img)
            img = img / 255.0
            batch_images.append(img)
        yield np.array(batch_images), np.array(batch_labels)
        
train_generator = generate_from_paths_and_labels(
    train_files, train_labels, BATCH_SIZE, train_datagen, IMG_SIZE)

val_generator = generate_from_paths_and_labels(
    val_files, val_labels, BATCH_SIZE, val_datagen, IMG_SIZE)

## Definición del modelo (versión ligera de VGG16)

In [7]:
def modified_lenet():
    model = models.Sequential([
        layers.Conv2D(6, (5, 5), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(16, (5, 5), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(120, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Salida binaria
    ])
    return model

## Creación y compilado del modelo

In [8]:
model = modified_lenet()
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 124, 6)       156       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 6)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 58, 58, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 29, 29, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 13456)             0         
                                                                 
 dense (Dense)               (None, 120)               1

## Entrenamiento

In [10]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_files) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_files) // BATCH_SIZE
)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

## Guardar el modelo

In [ ]:
model.save(models_dir('lenet5.h5'))